In [1]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch
import wandb
import fastai
from fastai.callback.wandb import WandbCallback

In [2]:
timm.list_models('*max*', pretrained=True)

['maxvit_base_tf_224.in1k',
 'maxvit_base_tf_384.in1k',
 'maxvit_base_tf_384.in21k_ft_in1k',
 'maxvit_base_tf_512.in1k',
 'maxvit_base_tf_512.in21k_ft_in1k',
 'maxvit_large_tf_224.in1k',
 'maxvit_large_tf_384.in1k',
 'maxvit_large_tf_384.in21k_ft_in1k',
 'maxvit_large_tf_512.in1k',
 'maxvit_large_tf_512.in21k_ft_in1k',
 'maxvit_nano_rw_256.sw_in1k',
 'maxvit_rmlp_base_rw_224.sw_in12k',
 'maxvit_rmlp_base_rw_224.sw_in12k_ft_in1k',
 'maxvit_rmlp_base_rw_384.sw_in12k_ft_in1k',
 'maxvit_rmlp_nano_rw_256.sw_in1k',
 'maxvit_rmlp_pico_rw_256.sw_in1k',
 'maxvit_rmlp_small_rw_224.sw_in1k',
 'maxvit_rmlp_tiny_rw_256.sw_in1k',
 'maxvit_small_tf_224.in1k',
 'maxvit_small_tf_384.in1k',
 'maxvit_small_tf_512.in1k',
 'maxvit_tiny_rw_224.sw_in1k',
 'maxvit_tiny_tf_224.in1k',
 'maxvit_tiny_tf_384.in1k',
 'maxvit_tiny_tf_512.in1k',
 'maxvit_xlarge_tf_384.in21k_ft_in1k',
 'maxvit_xlarge_tf_512.in21k_ft_in1k',
 'maxxvit_rmlp_nano_rw_256.sw_in1k',
 'maxxvit_rmlp_small_rw_256.sw_in1k',
 'maxxvitv2_nano_rw_2

In [3]:
from fastai.vision.all import *


def get_pets(batch_size, img_size, seed):
    "The dog breeds pets datasets"
    dataset_path = untar_data(URLs.PETS)
    files = get_image_files(dataset_path/"images")
    dls = ImageDataLoaders.from_name_re(dataset_path, files, 
                                        pat=r'(^[a-zA-Z]+_*[a-zA-Z]+)', 
                                        valid_pct=0.2, 
                                        seed=seed, 
                                        bs=batch_size,
                                        item_tfms=Resize(img_size)) 
    return dls

In [4]:
config = SimpleNamespace(
    batch_size=8,
    img_size=224,
    seed=42,
    pretrained=True,
    model_name="maxvit_rmlp_small_rw_224.sw_in1k", # try with maxvit_nano_rw_256.sw_in1k # regnetx_040 coatnet_bn_0_rw_224.sw_in1k
    epochs=5)


def train(config):
    "Train the model using the supplied config"
    dls = get_pets(config.batch_size, config.img_size, config.seed)
    with wandb.init(project="PROJECT", group='ambrosia_symbiosis', job_type='test_training', config=config):
        cbs = [MixedPrecision(), WandbCallback(log='all')]
        learn = vision_learner(dls, config.model_name, metrics=[error_rate, accuracy], 
                               cbs=cbs, pretrained=config.pretrained)
        learn.fine_tune(config.epochs)


train(config)

wandb: Currently logged in as: christopher-marais. Use `wandb login --relogin` to force relogin


C:\Users\gcmar\.conda\envs\BC_310\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.661420,0.352508,0.106901,0.893099,03:44


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.356592,0.258955,0.075778,0.924222,05:39
1,0.356859,0.213986,0.058863,0.941137,05:41
2,0.248149,0.205060,0.064953,0.935047,05:42
3,0.142010,0.189046,0.047361,0.952639,05:41
4,0.095533,0.173165,0.048038,0.951962,05:41


accuracy,▁▅▇▆██
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
error_rate,█▄▂▃▁▁
lr_0,▁▂▃▅▆▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_1,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
mom_0,█▇▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████
mom_1,█▇▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████
raw_loss,█▅▅▂▂▄▂▃▃▂▁▃▃▂▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▄▁▁▁▂▁▁▁▁▁▁
sqr_mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [5]:
def get_planets(batch_size=8, img_size=224, seed=42):
    "A sample of the planets dataset"
    dataset_path=untar_data(URLs.PLANET_SAMPLE)
    dls = ImageDataLoaders.from_csv(dataset_path, 
                                    folder="train", 
                                    csv_fname="labels.csv",
                                    label_delim=" ",
                                    suff=".jpg",
                                    bs=batch_size,
                                    seed=seed,
                                    item_tfms=Resize(img_size))
    return dls

In [6]:
get_planets()